<a href="https://colab.research.google.com/github/jjyothika15/aicte-internship-june-2025/blob/main/garbage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🗑️ Garbage Classification using EfficientNetV2B2
## Project Description
In this project, we aim to develop a sophisticated **garbage classification system** leveraging the **EfficientNetV2B2** architecture. Our primary dataset serves as a foundation for building models that can eventually automate waste segregation, a critical step in optimizing recycling and waste management, ultimately aiding in environmental conservation.

**Goal:** To develop an accurate and efficient garbage classification model using EfficientNetV2B2 and transfer learning for automated waste sorting.

## Challenges and Scope
**Key Challenge:** A notable challenge encountered is the inherent **class imbalance** within the dataset.


## Transfer Learning

Transfer Learning is a machine learning technique where a pre-trained model developed for a specific task is reused as the starting point for a model on a different but related task. This approach is beneficial when there is limited data for the new task, as the pre-trained model already has learned features that can be adapted.

### Benefits
- **Reduces training time**
- **Leverages learned features**
- **Improves performance** with less data

## How Does It Work?
1. Load a pretrained model (e.g., ResNet, EfficientNet).
2. Freeze the pretrained layers (optional).
3. Add new layers for your custom task.
4. Train on your new dataset (can also fine-tune).

---

## EfficientNetV2B2: Transfer Learning Backbone

EfficientNetV2B2 is a mid-sized model from the EfficientNetV2 family developed by Google.

### ⚙️ Key Features
- Fused MBConv blocks — enhance training stability and speed.
- Progressive learning — better generalization with fewer computations.
- Pretrained on ImageNet — ready for transfer learning.

### Why Use EfficientNetV2B2?

| Feature               | Description                              |
|----------------------|------------------------------------------|
| Balanced Performance | Good trade-off between speed and accuracy|
| Scalable             | Great for moderately sized datasets       |
| Efficient            | Trains faster, needs fewer resources     |


In [ ]:

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetV2B2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np


print("TensorFlow version:", tf.__version__)


In [ ]:

from google.colab import drive
drive.mount('/content/drive')


In [ ]:
dataset_path = '/content/drive/MyDrive/garbage_dataset'

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

train_gen = datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


In [ ]:
base_model = EfficientNetV2B2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
output = Dense(train_gen.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)


base_model.trainable = False

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


In [ ]:

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=5
)


In [ ]:

model.save('/content/efficientnetv2b2_garbage_classifier.h5')
print("✅ Model saved successfully!")


In [ ]:

plt.figure(figsize=(12,5))


plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()


plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()
